In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import warnings
from pandas.core.common import SettingWithCopyWarning

In [2]:
# Supress SettingWithCopy warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Import and Explore data:

In [3]:
# Import data from csv files
try:
    df_geo_1 = pd.read_csv('./geo_data_0.csv')
    df_geo_2 = pd.read_csv('./geo_data_1.csv')
    df_geo_3 = pd.read_csv('./geo_data_2.csv')
except:
    print("File(s) not found, please check file path(s) are correct")

File(s) not found, please check file path(s) are correct


In [4]:
df_geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
df_geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
df_geo_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
df_regions_list = [df_geo_1,df_geo_2,df_geo_3]
for df in df_regions_list:
    print(df.head(5))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [6]:
for df in df_regions_list:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [7]:
# Check for duplicate rows
for df in df_regions_list:
    print(df.duplicated().sum())

0
0
0


In [8]:
# Drop id column for each df
for df in df_regions_list:
    df.drop('id', axis=1, inplace=True)

In [9]:
# Get basic stats for each region
for df in df_regions_list:
    print(df.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

## Splitting Data and Model Training:

In [10]:
features_valid_all = pd.DataFrame(dtype='float64')
predictions_valid_all = pd.Series(dtype='float64')
target_valid_all = pd.Series(dtype='float64')

for region_index in range(len(df_regions_list)):
    
    df = df_regions_list[region_index]
    
    # Define feature and target
    features = df.drop(['product'], axis=1)
    target = df['product']
    
    # Use train_test_split() to split dataset into 75% training, 25% for validation
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    # Scale numerical columns using sklearn StandardScaler
    numeric_cols = ['f0', 'f1','f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric_cols])
    features_train[numeric_cols] = scaler.transform(features_train[numeric_cols])
    features_valid[numeric_cols] = scaler.transform(features_valid[numeric_cols])
    
    # Train linear regression model
    model = LinearRegression()
    model.fit(features_train,target_train) # train model on training set
    predictions_valid = model.predict(features_valid) # get model predictions on validation set
    mse = mean_squared_error(target_valid, predictions_valid)
    
    # Save features, predictions, and targets for validation sets for future use
    features_valid_all = pd.concat([features_valid_all, features_valid], ignore_index=True)
    predictions_valid_all = pd.concat([predictions_valid_all, pd.Series(predictions_valid)], ignore_index=True)
    target_valid_all = pd.concat([target_valid_all, target_valid], ignore_index=True)
    
    print("Region",region_index+1, ":")
    print('Average predicted volume:', np.mean(predictions_valid))
    print('MSE =', mse)
    print('RMSE:', mse**0.5)
    print('R2 =', r2_score(target_valid,predictions_valid))
    
    # Cross-validation of model using 10 blocks
    scores = cross_val_score(model, features, target, cv=10)
    final_score = np.mean(scores)
    print('Average model evaluation score:', final_score)
    
    print('')

Region 1 :
Average predicted volume: 92.59256778438035
MSE = 1412.2129364399243
RMSE: 37.5794217150813
R2 = 0.27994321524487786
Average model evaluation score: 0.27547024205606097

Region 2 :
Average predicted volume: 68.728546895446
MSE = 0.7976263360391157
RMSE: 0.893099286775617
R2 = 0.9996233978805127
Average model evaluation score: 0.9996243430392981

Region 3 :
Average predicted volume: 94.96504596800489
MSE = 1602.3775813236196
RMSE: 40.02970873393434
R2 = 0.20524758386040443
Average model evaluation score: 0.19867018805197603



The trained linear regression model performs best for region 2 with a low RMSE and an almost perfect R2 score. The average predicted well volume is close to the actual average volume for each region. The highest predicted average is for region 3, region 1 follows closely, while region 2 is significantly lower. 

Cross-validation of each model doesn't show any significant difference from initial result.

## Profit Calculation:

### Key Values:

In [11]:
# Budget for development of 200 oil wells is 100 USD million
budget = 100000000
# Revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels)
revenue_per_product_unit = 4500

budget_per_point = budget / 200

breakeven_volume_of_reserves = budget_per_point / revenue_per_product_unit

breakeven_volume_of_reserves

111.11111111111111

Minimum volume of well to be profitable is ~111,000 barrels, which is higher than the average predicted well volume for all regions

### Profit Calculation Function:

In [12]:
# Function that takes a list of well features to calculate potential profits using trained model predictions
def calculate_profit_for_well_list(wells):
    # initialize revenue
    revenue = 0
    wells_indeces = wells.index
    predicted_wells_volumes = predictions_valid_all[wells_indeces]
    predicted_wells_volumes_sorted = predicted_wells_volumes.sort_values(ascending=False)
    predicted_wells_volumes_top_200 = predicted_wells_volumes_sorted.head(200)
    for index, value in predicted_wells_volumes_top_200.iteritems():
        revenue+= target_valid_all[index]*revenue_per_product_unit
    profit = revenue - budget
    return int(profit)

In [13]:
# Function that takes region and random_state value to calculate profits from a sample of 500 points from validation set
def calculate_profit_for_wells_by_region(region_id, state):
    # Each region validation set contains 25000 rows, so we iterate through 25000 rows from dataframe containing all validation features
    step = int(features_valid_all.shape[0]/3)
    
    if region_id == 1:
        df = features_valid_all[0:step]
    elif region_id == 2: 
        df = features_valid_all[step:step*2]
    elif region_id == 3:    
        df = features_valid_all[step*2:step*3]
    
    # Pick 500 points for well exploration
    wells = df.sample(500, replace=True, random_state = state)    
    
    # Use calculate_profit_for_well_list() to calculate profits from wells with 200 highest predicted values from model
    return calculate_profit_for_well_list(wells)    

In [14]:
state = 12345

# Calculate profits for each region
for region in range(1,4):
    print("Region",region,": $", calculate_profit_for_wells_by_region(region, state))      

Region 1 : $ 6054640
Region 2 : $ 2280161
Region 3 : $ -718992


For random_state = 12345, region 1 has the most profit, but this is insufficient to decide which region is best

In [18]:
from collections import Counter

state = np.random.RandomState(12345)

# Run this calculation 1000 times and count how many times the region with the highest profit repeats
region_id_with_highest_profit_all_iterations = []
for iteration in range(1000):
    regions_profits = []
    for region in range(1,4):
        regions_profits.append(calculate_profit_for_wells_by_region(region, state))
    max_index = np.argmax(regions_profits)
    region_id_highest_profit = max_index + 1
    region_id_with_highest_profit_all_iterations.append(region_id_highest_profit)

counts = Counter(region_id_with_highest_profit_all_iterations)
print(counts)
most_common_value, count = counts.most_common(1)[0]
print(f"\nRegion with the highest profit simulations: {most_common_value} (Simulations with highest profit: {count})")

Counter({2: 398, 1: 302, 3: 300})

Region with the highest profit simulations: 2 (Simulations with highest profit: 398)


Region 2 consistenly has the highest chance of profit

## Risk and Profit by Region:

In [19]:
state = np.random.RandomState(12345)

for region in range(1,4):
    values = []
    for i in range(1000):
        subsample_profit = calculate_profit_for_wells_by_region(region, state)
        values.append(subsample_profit)

    values = pd.Series(values)
    lower = int(values.quantile(0.025))
    upper = int(values.quantile(0.975))
    mean = int(values.mean())
    
    print("Region",region, ":")
    print("Average profit:$", mean)
    print(f"95% confidence interval: (${lower}, ${upper})")
    print("% of iterations resulting in loss:", sum(values < 0)/len(values)*100)
    print()

Region 1 :
Average profit:$ 3961649
95% confidence interval: ($-1112155, $9097669)
% of iterations resulting in loss: 6.9

Region 2 :
Average profit:$ 4611557
95% confidence interval: ($780507, $8629520)
% of iterations resulting in loss: 0.7000000000000001

Region 3 :
Average profit:$ 3929504
95% confidence interval: ($-1122275, $9345628)
% of iterations resulting in loss: 6.5



## Conclusion:

Only region 2 has chance of loss less than 2.5% and it also has the highest average profit, so region 2 is recommended for further exploration.